In [1]:
from openai import OpenAI

import json
import os
import numpy as np
from tqdm import tqdm
from datasets import DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
import faiss

In [2]:
dataset = load_dataset('csv', data_files='..\data\dataset\processed\clean_data_gpt2.csv')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'essay', 'label'],
        num_rows: 9766
    })
})


split the train and test part for 9:1

In [3]:
split_dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'essay', 'label'],
        num_rows: 8789
    })
    test: Dataset({
        features: ['prompt', 'essay', 'label'],
        num_rows: 977
    })
})


Now, start to create embedding database

In [4]:
client = OpenAI()

In [6]:
def generate_embedding(text, model="text-embedding-ada-002"):
    try:
        response = client.embeddings.create(input=text, model=model)
        return response.data[0].embedding
    except Exception as e:
        print(f"Error generating embedding")
        return None

In [5]:
def generate_embeddings_batch(texts, model="text-embedding-ada-002"):
    try:
        response = client.embeddings.create(input=texts, model=model)
        return [item.embedding for item in response.data]
    except Exception as e:
        print(f"Error generating embeddings for batch: {e}")
        return None

In [7]:
embeddings = []
metadata = []


batch_size = 100  

for i in tqdm(range(0, len(split_dataset['train']), batch_size), desc="Generating embeddings"):
    batch_samples = split_dataset['train'][i:i+batch_size]

    
    
    combined_texts = [
        f"Prompt: {prompt}\nEssay: {essay}"
        for prompt, essay in zip(batch_samples['prompt'], batch_samples['essay'])
    ]
    
    batch_embeddings = generate_embeddings_batch(combined_texts)
    if batch_embeddings is not None:
        embeddings.extend(batch_embeddings)
        metadata.extend([
            {"prompt": prompt, "essay": essay, "label": label}
            for prompt, essay, label in zip(
                batch_samples['prompt'], batch_samples['essay'], batch_samples['label']
            )
        ])
    

# save the embeddings

embeddings_np = np.array(embeddings, dtype=np.float32)
faiss.normalize_L2(embeddings_np)  

dimension = len(embeddings_np[0])  
index = faiss.IndexFlatIP(dimension)  
index.add(embeddings_np)  

# save the reults
faiss.write_index(index, "faiss_index_train.bin")


with open("embeddings_dataset_train.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=4)

Generating embeddings: 100%|██████████| 88/88 [03:27<00:00,  2.35s/it]


RAG using topics

In [8]:
embeddings = []
metadata = []


batch_size = 100  

for i in tqdm(range(0, len(split_dataset['train']), batch_size), desc="Generating embeddings"):
    batch_samples = split_dataset['train'][i:i+batch_size]

    
    
    topics = [
        f"{prompt}"
        for prompt in batch_samples['prompt']
    ]
    
    batch_embeddings = generate_embeddings_batch(topics)
    if batch_embeddings is not None:
        embeddings.extend(batch_embeddings)
        metadata.extend([
            {"prompt": prompt, "essay": essay, "label": label}
            for prompt, essay, label in zip(
                batch_samples['prompt'], batch_samples['essay'], batch_samples['label']
            )
        ])
    

# save the embeddings

embeddings_np = np.array(embeddings, dtype=np.float32)
faiss.normalize_L2(embeddings_np)  

dimension = len(embeddings_np[0])  
index = faiss.IndexFlatIP(dimension)  
index.add(embeddings_np)  

# save the reults
faiss.write_index(index, "faiss_index_train_topics.bin")


with open("embeddings_dataset_train_topics.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=4)

Generating embeddings: 100%|██████████| 88/88 [01:06<00:00,  1.33it/s]


using test samples

In [9]:
embeddings = []
metadata = []


batch_size = 100  

for i in tqdm(range(0, len(split_dataset['test']), batch_size), desc="Generating embeddings"):
    batch_samples = split_dataset['test'][i:i+batch_size]

    
    
    topics = [
        f"{prompt}"
        for prompt in batch_samples['prompt']
    ]
    
    batch_embeddings = generate_embeddings_batch(topics)
    if batch_embeddings is not None:
        embeddings.extend(batch_embeddings)
        metadata.extend([
            {"prompt": prompt, "essay": essay, "label": label}
            for prompt, essay, label in zip(
                batch_samples['prompt'], batch_samples['essay'], batch_samples['label']
            )
        ])
    

# save the embeddings

embeddings_np = np.array(embeddings, dtype=np.float32)
faiss.normalize_L2(embeddings_np)  

dimension = len(embeddings_np[0])  
index = faiss.IndexFlatIP(dimension)  
index.add(embeddings_np)  

# save the reults
faiss.write_index(index, "faiss_index_test_topics.bin")


with open("embeddings_dataset_test_topics.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=4)

Generating embeddings: 100%|██████████| 10/10 [00:08<00:00,  1.15it/s]


In [10]:
index = faiss.read_index("faiss_index_train_topics.bin")
with open("embeddings_dataset_train_topics.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)


def search_cosine_similarity(query_text, top_k=3):
    
    query_embedding = generate_embedding(query_text)
    if query_embedding is None:
        return []
    
    
    query_embedding_np = np.array([query_embedding], dtype=np.float32)
    faiss.normalize_L2(query_embedding_np)
    
    
    distances, indices = index.search(query_embedding_np, top_k)
    
    
    results = []
    for i, idx in enumerate(indices[0]):
        result = metadata[idx]
        result["similarity"] = distances[0][i]  
        results.append(result)
    return results


query = """Many people use social media to keep in touch with other people and for news events. Do the advantages of this outweigh the disadvantages?
"""
results = search_cosine_similarity(query, top_k=3)

for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"  Prompt: {result['prompt']}")
    print(f"  Essay: {result['essay']}")
    print(f"  Label: {result['label']}")
    print(f"  Similarity: {result['similarity']}")
    print()

Result 1:
  Prompt: Many people use social media to keep in touch with other people and for news events. Do the advantages of this outweigh the disadvantages?
  Essay: In this day and age, the majority of people use social media to connect with each other and to receive  news. In my opinion, the benefits of using social platforms such as better communication and easy accessibility to news override the drawbacks of distractions and getting misleading information.

On the one hand, a large number of people use social networks to communicate with one another such as by using What's App or Facebook. This is a much easier and convenient way of having conversations with family members or friends especially when they are living in different cities or countries. For instance, international students can talk to their parents or siblings whenever they want through What's app video calls. Hence, it is extremely beneficial for those who live far away from their loved ones.

Furthermore, regarding 

In [ ]:
# embeddings = []
# metadata = []
# for sample in tqdm(split_dataset['train'], desc="Generating embeddings"):
#     combined_text = f"Prompt: {sample['prompt']}\nEssay: {sample['essay']}"
    
#     embedding = generate_embedding(combined_text)
#     if embedding is not None:
#         embeddings.append(embedding)
#         metadata.append({
#             "prompt": sample["prompt"],
#             "essay": sample["essay"],
#             "label": sample["label"]
#         })

                         

# # save the embeddings

# embeddings_np = np.array(embeddings, dtype=np.float32)
# faiss.normalize_L2(embeddings_np)  

# dimension = len(embeddings_np[0])  
# index = faiss.IndexFlatIP(dimension)  
# index.add(embeddings_np)  

# # save the reults
# faiss.write_index(index, "faiss_index.bin")


# with open("embeddings_dataset.json", "w", encoding="utf-8") as f:
#     json.dump(metadata, f, ensure_ascii=False, indent=4)



Generating embeddings:  15%|█▌        | 1359/8789 [04:35<19:43,  6.28it/s]  

In [5]:
import time
index = faiss.read_index("faiss_index_train.bin")
with open("embeddings_dataset_test.json", "r", encoding="utf-8") as f:
    metadata_test = json.load(f)

with open("embeddings_dataset_train.json", "r", encoding="utf-8") as f:
    metadata_train = json.load(f)

def search_cosine_similarity(query_text, dataset, top_k=3):
    
    query_embedding = generate_embedding(query_text)
    if query_embedding is None:
        return []
    
    
    query_embedding_np = np.array([query_embedding], dtype=np.float32)
    faiss.normalize_L2(query_embedding_np)
    
    
    distances, indices = index.search(query_embedding_np, top_k)
    
    
    results = []
    for i, idx in enumerate(indices[0]):
        result = dataset[idx]
        result["similarity"] = distances[0][i]  
        results.append(result)
    return results

correct = 0
cnt = 0
timeout_duration = 10
for data in metadata_test:
    query = f"Prompt: {data['prompt']}\nEssay: {data['essay']}"
    results = search_cosine_similarity(query, metadata_train, top_k=2)

    answer = float(data['label'])
    curr_score = 0
    weight = [0.8, 0.2]
    for i, result in enumerate(results):
        curr_score += result['label']*weight[i]

    # print(curr_score)
    print(cnt)


    if (abs(answer - curr_score) <= 1):
        correct += 1
    
    cnt += 1
    if cnt == 150:
        break
    # print("=====================")

print(f"Accuracy: {correct/cnt}")




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
Accuracy: 0.18666666666666668


# RAG with topic_essay combined similarity

create embedding database using essay content

In [7]:
embeddings = []
metadata = []


batch_size = 100  

for i in tqdm(range(0, len(split_dataset['train']), batch_size), desc="Generating embeddings"):
    batch_samples = split_dataset['train'][i:i+batch_size]

    
    
    essay = [
        f"{essay}"
        for essay in batch_samples['essay']
    ]
    
    batch_embeddings = generate_embeddings_batch(essay)
    if batch_embeddings is not None:
        embeddings.extend(batch_embeddings)
        metadata.extend([
            {"prompt": prompt, "essay": essay, "label": label}
            for prompt, essay, label in zip(
                batch_samples['prompt'], batch_samples['essay'], batch_samples['label']
            )
        ])
    

# save the embeddings

embeddings_np = np.array(embeddings, dtype=np.float32)
faiss.normalize_L2(embeddings_np)  

dimension = len(embeddings_np[0])  
index = faiss.IndexFlatIP(dimension)  
index.add(embeddings_np)  

# save the reults
faiss.write_index(index, "faiss_index_train_essay.bin")


with open("embeddings_dataset_train_essay.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=4)

Generating embeddings: 100%|██████████| 88/88 [03:00<00:00,  2.05s/it]


test

In [ ]:
index = faiss.read_index("faiss_index_train_essay.bin")
with open("embeddings_dataset_train_essay.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)


def search_cosine_similarity(query_text, top_k=3):
    
    query_embedding = generate_embedding(query_text)
    if query_embedding is None:
        return []
    
    
    query_embedding_np = np.array([query_embedding], dtype=np.float32)
    faiss.normalize_L2(query_embedding_np)
    
    
    distances, indices = index.search(query_embedding_np, top_k)
    
    
    results = []
    for i, idx in enumerate(indices[0]):
        result = metadata[idx]
        result["similarity"] = distances[0][i]  
        results.append(result)
    return results


query = """Many people use social media to keep in touch with other people and for news events. Do the advantages of this outweigh the disadvantages?
"""
results = search_cosine_similarity(query, top_k=3)

for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"  Prompt: {result['prompt']}")
    print(f"  Essay: {result['essay']}")
    print(f"  Label: {result['label']}")
    print(f"  Similarity: {result['similarity']}")
    print()